In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

code_postal_ville = [
    #Ile-de-France
    "75000",
    "75001", "75002", "75003", "75004", "75005", "75006", "75007", "75008", "75009", "75010",
    "75011", "75012", "75013", "75014", "75015", "75016", "75017", "75018", "75019", "75020",
    "92100", "92110", "92120", "92130", "92140", "92150", "92160", "92170", "92190",
    "92200", "92210", "92220", "92230", "92240", "92250", "92260", "92270", "92290",
    "92300", "92310", "92320", "92330", "92340", "92350", "92360", "92370", "92380", "92390",   "92400", "92410", "92420", "92430", "92440", "92450", "92460", "92470", "92480", "92490",
    "92500", "92510", "92520", "92530", "92540", "92550", "92560", "92570", "92580", "92590",
    "92600", "92610", "92620", "92630", "92640", "92650", "92660", "92670", "92680", "92690",
    "92700", "92710", "92720", "92730", "92740", "92750", "92760", "92770", "92780", "92790",
    "92800", "92810", "92820", "92830", "92840", "92850", "92860", "92870", "92880", "92890",
    "92900", "92910", "92920", "92930", "92940", "92950", "92960", "92970", "92980", "92990",
    "93000", "93100", "93200", "93300", "93400", "93500", "93600", "93700", "93800", "93900",
    "94000", "94100", "94200", "94300", "94400", "94500", "94600", "94700", "94800", "94900",
    
    #Marseille
    "13000", "13001", "13002", "13003", "13004", "13005", "13006", "13007", "13008", "13009",
    "13010", "13011", "13012", "13013",
    
    #Aix-en-Provence
    "13080", "13100", "13190", "13290", "13540", "13590", "13700", "13800", "13990",
    
    #Lyon
    "69000", "69001", "69002", "69003", "69004", "69005", "69006", "69007", "69008", "69009",
    "69100", "69200", "69300", "69400", "69500", "69600", "69700", "69800", "69900",
    
    #Lille
    "59000", "59100", "59200", "59300", "59400", "59500", "59600", "59700", "59800", "59900",
    
    #Bordeaux
    "33000", "33100", "33200", "33300", "33400", "33500", "33600", "33700", "33800", "33900",
    
    #Toulouse
    "31000", "31100", "31200", "31300", "31400", "31500", "31600", "31700", "31800", "31900",
    
    #Nice
    "06000", "06100", "06200", "06300", "06400", "06500", "06600", "06700", "06800", "06900",
    
    #Nantes
    "44000", "44100", "44200", "44300", "44400", "44500", "44600", "44700", "44800", "44900",
    
    #Strasbourg
    "67000", "67100", "67200", "67300", "67400", "67500", "67600", "67700", "67800", "67900",
    
    #Montpellier
    "34000", "34100", "34200", "34300", "34400", "34500", "34600", "34700", "34800", "34900",
    
    #Rennes
    "35000", "35100", "35200", "35300", "35400", "35500", "35600", "35700", "35800", "35900",
    
    #Grenoble
    "38000", "38100", "38200", "38300", "38400", "38500", "38600", "38700", "38800", "38900",
    
    #Dijon
    "21000", "21100", "21200", "21300", "21400", "21500", "21600", "21700", "21800", "21900",   
    
    #Angers
    "49000", "49100", "49200", "49300", "49400", "49500", "49600", "49700", "49800", "49900",
    
    #Rennes
    "35000", "35100", "35200", "35300", "35400", "35500", "35600", "35700", "35800", "35900",
    
    #Le Havre
    "76000", "76100", "76200", "76300", "76400", "76500", "76600", "76700", "76800", "76900",
    
    #Saint-Étienne
    "42000", "42100", "42200", "42300", "42400", "42500", "42600", "42700", "42800", "42900",
]

In [2]:
# === 1) Chemins des fichiers ===
communes_path = "Data/communes-france-2025.csv"
adj_path = "Data/communes_adjacentes_2022_toutes.csv"

# === 2) Charger les fichiers (en texte) ===
communes = pd.read_csv(communes_path, dtype=str)
adj = pd.read_csv(adj_path, dtype=str)

# === 3) Vérifier que les colonnes utiles existent ===
# communes: code INSEE, nom de la commune, codes postaux
if not all(c in communes.columns for c in ["code_insee", "nom_standard", "codes_postaux"]):
    raise ValueError("Le fichier communes doit contenir: code_insee, nom_standard, codes_postaux")

# adjacences: code INSEE source, voisins INSEE
if not all(c in adj.columns for c in ["insee", "insee_voisins"]):
    raise ValueError("Le fichier adjacences doit contenir: insee, insee_voisins")

# === 4) Petite fonction pour découper les listes (séparateur '|') ===
def split_pipe(value):
    """Retourne une liste en séparant par '|' (ou liste vide si NaN)."""
    if pd.isna(value):
        return []
    text = str(value).strip()
    if text == "":
        return []
    return [x.strip() for x in text.split("|") if x.strip()]

# === 5) Préparer la table 'communes' ===
# On garde seulement les colonnes utiles, et on "explose" les codes postaux
communes_simple = communes[["code_insee", "nom_standard", "codes_postaux"]].copy()
communes_simple["liste_cp"] = communes_simple["codes_postaux"].apply(split_pipe)
communes_cp = communes_simple.explode("liste_cp", ignore_index=True)  # une ligne par code postal
communes_cp = communes_cp.rename(columns={
    "nom_standard": "commune",
    "liste_cp": "code_postal"
})

# === 6) Préparer la table 'adjacences' ===
# On "explose" les voisins INSEE (une ligne par voisin)
adj["voisin_insee"] = adj["insee_voisins"].apply(split_pipe)
adj_long = adj.explode("voisin_insee", ignore_index=True)
adj_long = adj_long.dropna(subset=["voisin_insee"])  # garder seulement les lignes avec un voisin

# === 7) Joindre pour récupérer les infos de la banlieue (voisine) ===
# On va chercher, pour chaque voisin_insee, son nom et ses codes postaux
banlieue_infos = communes_simple.rename(columns={
    "code_insee": "code_insee_banlieue",
    "nom_standard": "banlieue",
    "codes_postaux": "codes_postaux_banlieue"
})

adj_avec_banlieue = adj_long.merge(
    banlieue_infos,
    left_on="voisin_insee",
    right_on="code_insee_banlieue",
    how="left"
)

# === 8) Joindre pour récupérer les infos de la commune source + son code postal (explosé) ===
# On relie le code INSEE source (adj["insee"]) au code INSEE des communes (communes_cp["code_insee"])
final = adj_avec_banlieue.merge(
    communes_cp[["code_insee", "commune", "code_postal"]],
    left_on="insee",
    right_on="code_insee",
    how="left"
)

# === 9) Ne garder que les colonnes finales, enlever doublons et lignes incomplètes ===
banlieues_df = final[[
    "code_postal",           # CP de la commune source
    "commune",               # Nom de la commune source
    "insee",                 # INSEE source
    "banlieue",              # Nom de la commune voisine (banlieue)
    "code_insee_banlieue",   # INSEE de la banlieue
    "codes_postaux_banlieue" # CP(s) de la banlieue (séparés par '|')
]].rename(columns={"insee": "code_insee_source"})

# On enlève les lignes où il manque l'essentiel
banlieues_df = banlieues_df.dropna(subset=["code_postal", "commune", "banlieue"])
banlieues_df = banlieues_df.drop_duplicates()

# === Filtrer une zone, ex. Île-de-France (75, 92, 93, 94) ===
banlieues_df = banlieues_df[banlieues_df["code_postal"].str.match(r"^(75|13|59|49|80|44|38)")]
banlieues_df.to_csv("banlieues_par_code_postal.csv", index=False, encoding="utf-8-sig")
# Afficher un aperçu
print(banlieues_df.head(10))

                                             code_postal          commune  \
28143  13092, 13602, 13616, 13791, 13798, 13628, 1379...  Aix-en-Provence   
28144  13092, 13602, 13616, 13791, 13798, 13628, 1379...  Aix-en-Provence   
28145  13092, 13602, 13616, 13791, 13798, 13628, 1379...  Aix-en-Provence   
28146  13092, 13602, 13616, 13791, 13798, 13628, 1379...  Aix-en-Provence   
28147  13092, 13602, 13616, 13791, 13798, 13628, 1379...  Aix-en-Provence   
28148  13092, 13602, 13616, 13791, 13798, 13628, 1379...  Aix-en-Provence   
28149  13092, 13602, 13616, 13791, 13798, 13628, 1379...  Aix-en-Provence   
28150  13092, 13602, 13616, 13791, 13798, 13628, 1379...  Aix-en-Provence   
28151  13092, 13602, 13616, 13791, 13798, 13628, 1379...  Aix-en-Provence   
28152  13092, 13602, 13616, 13791, 13798, 13628, 1379...  Aix-en-Provence   

      code_insee_source                banlieue code_insee_banlieue  \
28143             13001  Le Puy-Sainte-Réparade               13080   
28144     

dataset des banlieues entourant des codes postaux en France

In [3]:
# ---  Normaliser les codes postaux  ---
banlieues_df["code_postal"] = (
    banlieues_df["code_postal"]
    .astype(str)
    .str.extract(r"(\d{2,5})", expand=False)  # récupère les chiffres principaux
    .fillna("")
    .str.zfill(5)
)

# ---  Filtrer : ne garder que les lignes dont le CP SOURCE est dans ta liste ---
cp_set = set(code_postal_ville)
extrait_source = banlieues_df[banlieues_df["code_postal"].isin(cp_set)].copy()

# ---  inclure aussi si la BANLIEUE possède un CP dans ta liste ---
def banlieue_a_cp_dans_liste(cell, cp_set):
    """Retourne True si au moins un CP (séparés par '|') de la banlieue est dans code_postal_ville."""
    if pd.isna(cell) or str(cell).strip() == "":
        return False
    return any(cp.strip().zfill(5) in cp_set for cp in str(cell).split("|"))

mask_banlieue = banlieues_df["codes_postaux_banlieue"].apply(lambda x: banlieue_a_cp_dans_liste(x, cp_set))
extrait_banlieue = banlieues_df[mask_banlieue].copy()

extrait = extrait_source


# ---  Garder les colonnes utiles + dédoublonner + trier ---
colonnes_utiles = [
    "code_postal",
    "commune",
    "banlieue",
    "code_insee_source",
    "code_insee_banlieue",
    "codes_postaux_banlieue",
]
extrait = (
    extrait[colonnes_utiles]
    .dropna(subset=["code_postal", "commune", "banlieue"])
    .drop_duplicates()
    .sort_values(["code_postal", "commune", "banlieue"], kind="stable")
    .reset_index(drop=True)
)
# --- export --- 
print("Lignes retenues :", len(extrait))
display(extrait.head(10))

outfile = "banlieues_extrait_selon_liste.csv"
extrait.to_csv(outfile, index=False, encoding="utf-8-sig")
print("Fichier exporté :",outfile)



Lignes retenues : 431


,code_postal,commune,banlieue,code_insee_source,code_insee_banlieue,codes_postaux_banlieue
0,13100,Beaurecueil,Châteauneuf-le-Rouge,13012,13025,13790
1,13100,Beaurecueil,Le Tholonet,13012,13109,"13100, 13182, 13097, 13603"
2,13100,Beaurecueil,Meyreuil,13012,13060,13590
3,13100,Beaurecueil,Saint-Antonin-sur-Bayon,13012,13090,13100
4,13100,Beaurecueil,Saint-Marc-Jaumegarde,13012,13095,13100
5,13100,Beaurecueil,Vauvenargues,13012,13111,13126
6,13100,Le Tholonet,Aix-en-Provence,13109,13001,"13092, 13602, 13616, 13791, 13798, 13628, 1379..."
7,13100,Le Tholonet,Beaurecueil,13109,13012,13100
8,13100,Le Tholonet,Meyreuil,13109,13060,13590
9,13100,Le Tholonet,Saint-Marc-Jaumegarde,13109,13095,13100


Fichier exporté : banlieues_extrait_selon_liste.csv


In [ ]:
import ipywidgets as w
from IPython.display import display, clear_output

# Prépare département à partir du CP
extrait["departement"] = extrait["code_postal"].astype(str).str[:2]

tabs = w.ToggleButtons(options=[("Top banlieues","ban"), ("Top communes","com")], value="ban")
dept_opts = sorted(extrait["departement"].dropna().unique().tolist())
dept_sel = w.SelectMultiple(options=dept_opts, description="Dépts", rows=min(10, len(dept_opts)))
search = w.Text(value="", description="Contient:", placeholder="nom à chercher (commune ou banlieue)")
topn = w.IntSlider(value=15, min=5, max=50, step=1, description="Top N", continuous_update=False)
out = w.Output()

def filtered_df():
    df = extrait.copy()
    # filtre département
    if dept_sel.value:
        df = df[df["departement"].isin(list(dept_sel.value))]
    # recherche texte
    q = search.value.strip()
    if q:
        mask = df["commune"].str.contains(q, case=False, na=False) | df["banlieue"].str.contains(q, case=False, na=False)
        df = df[mask]
    return df

def refresh(_=None):
    with out:
        clear_output(wait=True)
        df = filtered_df()
        if df.empty:
            print("Aucune donnée après filtres.")
            return
        if tabs.value == "ban":
            vc = df["banlieue"].dropna().astype(str).value_counts().head(topn.value).sort_values(ascending=True)
            plt.figure(figsize=(10,6)); plt.barh(vc.index, vc.values)
            plt.xlabel("Occurrences"); plt.title(f"Top {topn.value} banlieues (filtres appliqués)")
            plt.grid(axis="x", linestyle="--", alpha=0.4); plt.tight_layout(); plt.show()
        else:
            vc = df["commune"].dropna().astype(str).value_counts().head(topn.value).sort_values(ascending=True)
            plt.figure(figsize=(10,6)); plt.barh(vc.index, vc.values)
            plt.xlabel("Nb de banlieues reliées"); plt.title(f"Top {topn.value} communes (filtres appliqués)")
            plt.grid(axis="x", linestyle="--", alpha=0.4); plt.tight_layout(); plt.show()

for wgt in [tabs, dept_sel, search, topn]:
    wgt.observe(refresh, names="value")

display(w.VBox([w.HBox([tabs, topn]), w.HBox([dept_sel, search]), out]))
refresh()


In [5]:
import ipywidgets as w
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# --- Prépare département à partir du CP ---
extrait = extrait.copy()
extrait["departement"] = extrait["code_postal"].astype(str).str[:2]

tabs = w.ToggleButtons(options=[("Top banlieues","ban"), ("Top communes","com")], value="ban")
dept_opts = sorted(extrait["departement"].dropna().unique().tolist())
dept_sel = w.SelectMultiple(options=dept_opts, description="Dépts", rows=min(10, len(dept_opts)))
search = w.Text(value="", description="Contient:", placeholder="nom à chercher (commune ou banlieue)")
topn = w.IntSlider(value=15, min=5, max=50, step=1, description="Top N", continuous_update=False)

# NEW: type de graphe
chart_type = w.Dropdown(
    options=[
        ("Barres horizontales", "barh"),
        ("Lollipop (tige + point)", "lollipop"),
        ("Dot plot (points)", "dot"),
        ("Donut (anneau)", "donut"),
        ("Pareto (barres + % cum.)", "pareto"),
    ],
    value="barh",
    description="Graphique:"
)

out = w.Output()

def filtered_df():
    df = extrait.copy()
    if dept_sel.value:
        df = df[df["departement"].isin(list(dept_sel.value))]
    q = search.value.strip()
    if q:
        mask = df["commune"].str.contains(q, case=False, na=False) | df["banlieue"].str.contains(q, case=False, na=False)
        df = df[mask]
    return df

def make_top_series(df, kind, n):
    col = "banlieue" if kind == "ban" else "commune"
    vc = df[col].dropna().astype(str).value_counts().head(n)
    # pour tous les graphes, on trie ascendant pour barh/lollipop/dot (bas → haut)
    vc = vc.sort_values(ascending=True)
    return vc, col

def plot_barh(labels, values, title, xlab):
    plt.figure(figsize=(10,6))
    plt.barh(labels, values)
    for i, v in enumerate(values):
        plt.text(v, i, f" {v}", va="center")
    plt.xlabel(xlab)
    plt.title(title)
    plt.grid(axis="x", linestyle="--", alpha=0.4)
    plt.tight_layout()
    plt.show()

def plot_lollipop(labels, values, title, xlab):
    y = np.arange(len(labels))
    plt.figure(figsize=(10,6))
    # tiges
    plt.hlines(y=y, xmin=0, xmax=values, alpha=0.6)
    # points
    plt.plot(values, y, "o")
    for i, v in enumerate(values):
        plt.text(v, y[i], f" {v}", va="center")
    plt.yticks(y, labels)
    plt.xlabel(xlab)
    plt.title(title)
    plt.grid(axis="x", linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_dot(labels, values, title, xlab):
    y = np.arange(len(labels))
    plt.figure(figsize=(10,6))
    plt.scatter(values, y, s=60)  # points seuls
    for i, v in enumerate(values):
        plt.text(v, y[i], f" {v}", va="center")
    plt.yticks(y, labels)
    plt.xlabel(xlab)
    plt.title(title)
    plt.grid(axis="x", linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_donut(labels, values, title):
    # donut lisible pour ~10 items max
    if len(values) > 10:
        labels = labels[-10:]
        values = values[-10:]
    plt.figure(figsize=(7,7))
    wedges, texts, autotexts = plt.pie(
        values,
        labels=labels,
        autopct="%1.1f%%",
        startangle=140,
        wedgeprops={"width":0.4, "edgecolor":"white"}
    )
    plt.title(title)
    plt.tight_layout()
    plt.show()

def plot_pareto(labels, values, title):
    vals = np.array(values)
    labels = list(labels)
    # pour Pareto, on veut barres triées décroissantes
    order = np.argsort(vals)[::-1]
    vals = vals[order]
    labels = [labels[i] for i in order]
    cum = np.cumsum(vals) / vals.sum() * 100.0

    fig, ax1 = plt.subplots(figsize=(10,6))
    ax1.bar(range(len(vals)), vals)
    ax1.set_xlabel("Éléments")
    ax1.set_ylabel("Occurrences")
    ax1.set_xticks(range(len(vals)))
    ax1.set_xticklabels(labels, rotation=45, ha="right")

    ax2 = ax1.twinx()
    ax2.plot(range(len(vals)), cum, marker="o", linewidth=2)
    ax2.set_ylabel("% cumulé")
    ax2.set_ylim(0, 110)
    ax2.axhline(80, color="gray", linestyle="--", alpha=0.5)  # repère 80%
    plt.title(title)
    fig.tight_layout()
    plt.show()

def refresh(_=None):
    with out:
        clear_output(wait=True)
        df = filtered_df()
        if df.empty:
            print("Aucune donnée après filtres.")
            return

        vc, col = make_top_series(df, tabs.value, topn.value)
        if vc.empty:
            print("Aucune donnée à afficher.")
            return

        title = f"Top {len(vc)} {'banlieues' if col=='banlieue' else 'communes'} (filtres appliqués)"
        xlab = "Occurrences"

        labels = list(vc.index)
        values = list(vc.values)

        if chart_type.value == "barh":
            plot_barh(labels, values, title, xlab)
        elif chart_type.value == "lollipop":
            plot_lollipop(labels, values, title, xlab)
        elif chart_type.value == "dot":
            plot_dot(labels, values, title, xlab)
        elif chart_type.value == "donut":
            plot_donut(labels, values, title)
        elif chart_type.value == "pareto":
            plot_pareto(labels, values, title)

for wgt in [tabs, dept_sel, search, topn, chart_type]:
    wgt.observe(refresh, names="value")

display(w.VBox([w.HBox([tabs, topn, chart_type]), w.HBox([dept_sel, search]), out]))
refresh()


In [6]:
# ==== DEBUG & AFFICHAGE ROBUSTE FOLIUM ====
import numpy as np
import folium
from folium.plugins import MarkerCluster
from IPython.display import display, HTML, IFrame, FileLink

# 0) Vérifs rapides
print("ex shape:", ex.shape if 'ex' in globals() else "ex n'existe pas")
need_cols = ["lat_src","lon_src","lat_dst","lon_dst"]
missing = [c for c in need_cols if c not in ex.columns]
if missing:
    print("Colonnes manquantes pour la carte:", missing)

# 1) Convertir en numérique (obligatoire pour Folium)
for c in ["lat_src","lon_src","lat_dst","lon_dst"]:
    if c in ex.columns:
        ex[c] = pd.to_numeric(ex[c], errors="coerce")

# 2) Filtrer lignes traçables
ex_geo = ex.dropna(subset=["lat_src","lon_src","lat_dst","lon_dst"]).copy()
print("Lignes traçables (ex_geo):", len(ex_geo))
if ex_geo.empty:
    raise ValueError("Aucune ligne traçable : vérifie que tes colonnes latitude/longitude existent et contiennent des valeurs.")

# 3) Centre carte
latc = np.nanmean(pd.concat([ex_geo["lat_src"], ex_geo["lat_dst"]]))
lonc = np.nanmean(pd.concat([ex_geo["lon_src"], ex_geo["lon_dst"]]))
if np.isnan(latc) or np.isnan(lonc):
    latc, lonc = 46.6, 2.5  # centre France par défaut

# 4) Construire la carte
m = folium.Map(location=(float(latc), float(lonc)), zoom_start=7, tiles="cartodbpositron")

cluster_src = MarkerCluster(name="Communes (sources)").add_to(m)
cluster_dst = MarkerCluster(name="Banlieues (voisines)").add_to(m)

# Marqueurs sources
for _, r in ex_geo.drop_duplicates(subset=["code_insee_source"]).iterrows():
    folium.CircleMarker(
        location=(r["lat_src"], r["lon_src"]),
        radius=6, color="#2c7fb8", fill=True, fill_opacity=0.9,
        popup=folium.Popup(f"<b>{r['commune']}</b><br>CP: {r['code_postal']}<br>INSEE: {r['code_insee_source']}", max_width=260),
        tooltip=f"{r['commune']} (source)"
    ).add_to(cluster_src)

# Marqueurs banlieues
for _, r in ex_geo.drop_duplicates(subset=["code_insee_banlieue"]).iterrows():
    label = r.get("banlieue_nom", r["banlieue"])
    folium.CircleMarker(
        location=(r["lat_dst"], r["lon_dst"]),
        radius=5, color="#f03b20", fill=True, fill_opacity=0.9,
        popup=folium.Popup(f"<b>{label}</b><br>INSEE: {r['code_insee_banlieue']}", max_width=260),
        tooltip=f"{label} (banlieue)"
    ).add_to(cluster_dst)

# Lignes
for _, r in ex_geo.iterrows():
    folium.PolyLine(
        locations=[(r["lat_src"], r["lon_src"]), (r["lat_dst"], r["lon_dst"])],
        color="#636363", weight=1.2, opacity=0.6
    ).add_to(m)

folium.LayerControl().add_to(m)

# 5) AFFICHAGE — 3 méthodes (au moins une marchera)
print("Tentative 1: display(m)")
display(m)





ModuleNotFoundError: No module named 'folium'

In [ ]:


# Charger le fichier téléchargé depuis data.gouv
epci_path = "Composition_epci_2025.csv"
epci = pd.read_csv(epci_path, dtype=str)

# Filtrer uniquement les métropoles
metropoles = epci[epci["TYPE EPCI"].str.contains("Métropole", case=False, na=False)].copy()

# Garder seulement les colonnes utiles
metropoles = metropoles.rename(columns={
    "NOM DE L’EPCI": "metropole",
    "NOM DE LA COMMUNE": "commune",
    "CODE INSEE": "code_insee",
    "SIEGE DE L’EPCI": "siege"
})[["code_insee", "commune", "metropole", "siege"]]

print("Nombre de communes appartenant à une métropole :", len(metropoles))
display(metropoles.head(10))





In [ ]:
import pandas as pd

# =========================
# 1) Paramètres d'entrée
# =========================
cp_list = code_postal_ville  # ta liste fournie
cp_set = set(str(cp).zfill(5) for cp in cp_list)

EPCI_PATH = "Data/Composition_epci_2025.csv"  # fourni
NE_GARDER_QUE_METROPOLES = True  # mets False si tu ne veux pas ce filtre

# =========================
# 2) Utilitaires
# =========================
def normalize_cp_series(s: pd.Series) -> pd.Series:
    """Garde 2 à 5 chiffres, complète à 5, remplace NaN par ''."""
    return (
        s.astype(str)
         .str.extract(r"(\d{2,5})", expand=False)
         .fillna("")
         .str.zfill(5)
    )

def has_cp_in_list(cell, cp_set) -> bool:
    """
    True si au moins un CP (séparés par '|', espaces ou virgules) apparait dans cp_set.
    """
    if pd.isna(cell):
        return False
    txt = str(cell).strip()
    if not txt:
        return False
    # tolère séparateurs : | , ; espace
    parts = []
    for sep in ["|", ",", ";"]:
        if sep in txt:
            parts = [p.strip() for p in txt.split(sep)]
            break
    if not parts:
        # sépare aussi sur espaces multiples
        parts = [p for p in txt.replace("  ", " ").split(" ")]
    return any(str(p).strip().zfill(5) in cp_set for p in parts if str(p).strip())

def load_insee_communes_metropoles(epci_csv_path: str) -> set:
    """
    Charge le CSV Composition EPCI et renvoie l'ensemble des codes INSEE des communes
    qui appartiennent à un EPCI de type 'Métropole' (insensible à la casse/accents).
    Essaie plusieurs en-têtes possibles.
    """
    df = pd.read_csv(epci_csv_path, dtype=str)
    # Colonnes possibles selon les sources
    cols = {c.lower().strip(): c for c in df.columns}

    # repère la colonne type epci
    type_col = None
    for key in ["type epci", "type_epci", "type"]:
        if key in cols:
            type_col = cols[key]
            break
    if type_col is None:
        raise KeyError("Colonne 'TYPE EPCI' introuvable dans le fichier EPCI.")

    # repère la colonne code insee commune
    insee_col = None
    for key in ["code insee commune", "code_insee_commune", "insee_commune", "code insee"]:
        if key in cols:
            insee_col = cols[key]
            break
    if insee_col is None:
        raise KeyError("Colonne 'CODE INSEE COMMUNE' introuvable dans le fichier EPCI.")

    # filtre métropoles (tolère accents et casse)
    mask_metro = df[type_col].fillna("").str.normalize("NFKD").str.encode("ascii", "ignore").str.decode("ascii")
    mask_metro = mask_metro.str.contains("metropole", case=False, na=False)

    insee_set = set(df.loc[mask_metro, insee_col].dropna().str.strip())
    # normalise sur 5 caractères (certains INSEE peuvent avoir 5/6 avec Corses)
    insee_set = {x[:5] for x in insee_set if x}
    return insee_set

# =========================
# 3) Préparation des données banlieues
# =========================
# Colonnes attendues dans banlieues_df :
#   - "code_postal" (source)
#   - "commune"
#   - "banlieue"
#   - "code_insee_source"
#   - "code_insee_banlieue"
#   - "codes_postaux_banlieue" (CP multiples possibles)
# Adapte ci-dessous si tes noms diffèrent.
required_cols = [
    "code_postal", "commune", "banlieue",
    "code_insee_source", "code_insee_banlieue", "codes_postaux_banlieue"
]
missing = [c for c in required_cols if c not in banlieues_df.columns]
if missing:
    raise KeyError(f"Colonnes manquantes dans banlieues_df : {missing}")

# normalise CP source
banlieues_df = banlieues_df.copy()
banlieues_df["code_postal"] = normalize_cp_series(banlieues_df["code_postal"])

# =========================
# 4) Filtres par CP (source + banlieue)
# =========================
mask_source_cp = banlieues_df["code_postal"].isin(cp_set)
mask_banlieue_cp = banlieues_df["codes_postaux_banlieue"].apply(lambda x: has_cp_in_list(x, cp_set))

mask_cp_union = mask_source_cp | mask_banlieue_cp
filtered = banlieues_df[mask_cp_union].copy()

# =========================
# 5) (Optionnel) Filtre Métropoles via INSEE
# =========================
if NE_GARDER_QUE_METROPOLES:
    try:
        insee_metro = load_insee_communes_metropoles(EPCI_PATH)

        # normaliser INSEE sur 5 chars (au cas où)
        filtered["code_insee_source"] = filtered["code_insee_source"].astype(str).str.strip().str[:5]
        filtered["code_insee_banlieue"] = filtered["code_insee_banlieue"].astype(str).str.strip().str[:5]

        mask_insee_metro = (
            filtered["code_insee_source"].isin(insee_metro) |
            filtered["code_insee_banlieue"].isin(insee_metro)
        )
        filtered = filtered[mask_insee_metro].copy()
    except Exception as e:
        print("⚠️ Filtre Métropoles non appliqué (problème de lecture EPCI) :", e)

# =========================
# 6) Colonnes utiles, dédoublonnage, tri, export
# =========================
colonnes_utiles = [
    "code_postal",
    "commune",
    "banlieue",
    "code_insee_source",
    "code_insee_banlieue",
    "codes_postaux_banlieue",
]

extrait = (
    filtered[colonnes_utiles]
    .dropna(subset=["code_postal", "commune", "banlieue"])
    .drop_duplicates()
    .sort_values(["code_postal", "commune", "banlieue"], kind="stable")
    .reset_index(drop=True)
)

print("Lignes retenues :", len(extrait))
display(extrait.head(10))

outfile = "banlieues_extrait_selon_liste_metropoles.csv" if NE_GARDER_QUE_METROPOLES else "banlieues_extrait_selon_liste.csv"
extrait.to_csv(outfile, index=False, encoding="utf-8-sig")
print("Fichier exporté :", outfile)
